## Introduction: Carbon Tax

By definition, carbon tax is: \
\
_A tax levied on the carbon emissions from producing goods and services. Carbon taxes are intended to make visible the hidden social costs of carbon emissions. They are designed to reduce greenhouse gas emissions by essentially increasing the price of fossil fuels._

To do this, we will assume some typical values (these are stylized but realistic order of magnitude for thermal plants; renewables are ~0 in dispatch terms):
- Wind:      0.0 tCO₂/MWh
- Nuclear:   0.0 tCO₂/MWh (operationally)
- Biomass:   0.0 tCO₂/MWh (we’ll treat biomass as carbon neutral in dispatch, many models do this; in reality it’s debated)
- Coal:      0.9 tCO₂/MWh
- Gas:       0.4 tCO₂/MWh)

In [1]:
import pulp as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

We will still be using our predefined function

In [ ]:
def solve_power_system_with_carbon(capacity, demand_region, carbon_price):
    regions = ["North", "Central", "South"]

    plants = [
        "Wind_1",
        "Wind_2",
        "Nuclear_1",
        "Biomass_1",
        "Coal_1",
        "Gas_1",
        "Gas_2"
    ]

    plant_region = {
        "Wind_1": "North",
        "Nuclear_1": "North",
        "Wind_2": "Central",
        "Biomass_1": "Central",
        "Coal_1": "South",
        "Gas_1": "South",
        "Gas_2": "South"
    }

    # Base marginal cost [€/MWh] (fuel+variable O&M without CO2 price)
    marginal_cost = {
        "Wind_1": 0,
        "Wind_2": 0,
        "Nuclear_1": 10,
        "Biomass_1": 40,
        "Coal_1": 60,
        "Gas_1": 80,
        "Gas_2": 100
    }

    # Emissions factor [tCO2 / MWh]
    emissions = {
        "Wind_1": 0.0,
        "Wind_2": 0.0,
        "Nuclear_1": 0.0,
        "Biomass_1": 0.0,   # assumption: carbon-neutral in dispatch
        "Coal_1": 0.9,
        "Gas_1": 0.4,
        "Gas_2": 0.4
    }

    # Effective marginal cost including CO2 (€/MWh)
    effective_cost = {
        p: marginal_cost[p] + carbon_price * emissions[p]
        for p in plants
    }

    # Transmission arcs (directed), with symmetric or asymmetric limits
    arcs = [
        ("North", "Central"),
        ("Central", "North"),
        ("Central", "South"),
        ("South", "Central")
    ]

    line_capacity = {
        ("North", "Central"): 200,
        ("Central", "North"): 200,
        ("Central", "South"): 150,
        ("South", "Central"): 150
    }

    # Build the model
    model = pl.LpProblem("network_dispatch_with_carbon", pl.LpMinimize)

    # Generation decision variables [MW]
    gen = {
        p: pl.LpVariable(
            f"Gen_{p}",
            lowBound=0,
            upBound=capacity[p],
            cat="Continuous"
        )
        for p in plants
    }

    # Flow decision variables [MW]
    flow = {
        (src, dst): pl.LpVariable(
            f"Flow_{src}_to_{dst}",
            lowBound=0,
            upBound=line_capacity[(src, dst)],
            cat="Continuous"
        )
        for (src, dst) in arcs
    }

    # Objective: minimize total system operating cost incl. CO2 penalty
    model += pl.lpSum(effective_cost[p] * gen[p] for p in plants)

    # Regional nodal balance
    for region in regions:
        gen_in_region = pl.lpSum(
            gen[p] for p in plants if plant_region[p] == region
        )

        inflow_region = pl.lpSum(
            flow[(src, dst)]
            for (src, dst) in arcs
            if dst == region
        )

        outflow_region = pl.lpSum(
            flow[(src, dst)]
            for (src, dst) in arcs
            if src == region
        )

        model += (
            gen_in_region + inflow_region - outflow_region
            == demand_region[region]
        ), f"Balance_{region}"

    # Solve
    model.solve(pl.PULP_CBC_CMD(msg=False))

    # Generating Reports
    print(f"Status: {pl.LpStatus[model.status]}")
    system_cost = pl.value(model.objective)
    print(f"Total System Cost (incl. CO2): €{system_cost:.2f}\n")

    print(
        "Effective marginal cost [€/MWh] with CO2 price {} €/t:".format(carbon_price))
    for p in plants:
        print(f"  {p:<10} : {effective_cost[p]:6.2f} €/MWh")

    print("\nDispatch by plant [MW]:")
    for p in plants:
        print(
            f"  {p:<10} : {gen[p].value():6.2f} / {capacity[p]} MW "
            f"(region {plant_region[p]})"
        )

    print("\nFlows between regions [MW]:")
    for (src, dst) in arcs:
        val = flow[(src, dst)].value()
        if val > 1e-6:
            print(
                f"  {src} → {dst}: {val:.2f} MW "
                f"(limit {line_capacity[(src, dst)]} MW)"
            )

    print("\nRegional supply/demand check:")
    for region in regions:
        gen_val = sum(
            gen[p].value() for p in plants if plant_region[p] == region
        )
        inflow_val = sum(
            flow[(src, dst)].value()
            for (src, dst) in arcs
            if dst == region
        )
        outflow_val = sum(
            flow[(src, dst)].value()
            for (src, dst) in arcs
            if src == region
        )
        lhs = gen_val + inflow_val - outflow_val
        print(
            f"  {region:<7} demand {demand_region[region]:6.2f}  |  "
            f"LHS {lhs:6.2f}  "
            f"(gen {gen_val:.2f} + inflow {inflow_val:.2f} - outflow {outflow_val:.2f})"
        )

    return model

In [3]:
demand_stress = {
    "North":   300,
    "Central": 250,
    "South":   300
}

cap_upgrade = {
    "Wind_1": 100,
    "Wind_2": 100,
    "Nuclear_1": 300,   # upgraded nuclear in North
    "Biomass_1": 150,
    "Coal_1": 250,
    "Gas_1": 150,
    "Gas_2": 150
}

print("=== No CO2 Price ===")
solve_power_system_with_carbon(cap_upgrade, demand_stress, carbon_price=0)

print("\n=== CO2 Price 50 €/t ===")
solve_power_system_with_carbon(cap_upgrade, demand_stress, carbon_price=50)

=== No CO2 Price ===
Status: Optimal
Total System Cost (incl. CO2): €21000.00

Effective marginal cost [€/MWh] with CO2 price 0 €/t:
  Wind_1     :   0.00 €/MWh
  Wind_2     :   0.00 €/MWh
  Nuclear_1  :  10.00 €/MWh
  Biomass_1  :  40.00 €/MWh
  Coal_1     :  60.00 €/MWh
  Gas_1      :  80.00 €/MWh
  Gas_2      : 100.00 €/MWh

Dispatch by plant [MW]:
  Wind_1     : 100.00 / 100 MW (region North)
  Wind_2     : 100.00 / 100 MW (region Central)
  Nuclear_1  : 300.00 / 300 MW (region North)
  Biomass_1  : 150.00 / 150 MW (region Central)
  Coal_1     : 200.00 / 250 MW (region South)
  Gas_1      :   0.00 / 150 MW (region South)
  Gas_2      :   0.00 / 150 MW (region South)

Flows between regions [MW]:
  North → Central: 100.00 MW (limit 200 MW)
  Central → South: 100.00 MW (limit 150 MW)

Regional supply/demand check:
  North   demand 300.00  |  LHS 300.00  (gen 400.00 + inflow 0.00 - outflow 100.00)
  Central demand 250.00  |  LHS 250.00  (gen 250.00 + inflow 100.00 - outflow 100.00)
  

network_dispatch_with_carbon:
MINIMIZE
40.0*Gen_Biomass_1 + 105.0*Gen_Coal_1 + 100.0*Gen_Gas_1 + 120.0*Gen_Gas_2 + 10.0*Gen_Nuclear_1 + 0.0
SUBJECT TO
Balance_North: Flow_Central_to_North - Flow_North_to_Central + Gen_Nuclear_1
 + Gen_Wind_1 = 300

Balance_Central: - Flow_Central_to_North - Flow_Central_to_South
 + Flow_North_to_Central + Flow_South_to_Central + Gen_Biomass_1 + Gen_Wind_2
 = 250

Balance_South: Flow_Central_to_South - Flow_South_to_Central + Gen_Coal_1
 + Gen_Gas_1 + Gen_Gas_2 = 300

VARIABLES
Flow_Central_to_North <= 200 Continuous
Flow_Central_to_South <= 150 Continuous
Flow_North_to_Central <= 200 Continuous
Flow_South_to_Central <= 150 Continuous
Gen_Biomass_1 <= 150 Continuous
Gen_Coal_1 <= 250 Continuous
Gen_Gas_1 <= 150 Continuous
Gen_Gas_2 <= 150 Continuous
Gen_Nuclear_1 <= 300 Continuous
Gen_Wind_1 <= 100 Continuous
Gen_Wind_2 <= 100 Continuous

### The logic behind:

As you can see, we have updated the objective into:

effective_cost = {
    p: marginal_cost[p] + carbon_price * emissions[p]
    for p in plants
}

model += pl.lpSum(effective_cost[p] * gen[p] for p in plants)

That means, coal price just jumped from 60 + (50 * 0.9 = 45), so practically 105 EUR per kWh ! \
This is more expensive than Gas 1, even though still cheaper than Gas 2 (both also includes carbon tax, now) \


Hence, the dispatch should shift:
- Coal becomes way more expensive.
- Nuclear + imports should push coal down.
- South might even import more instead of running coal hard.
- If nuclear isn’t enough to fully cover South, we might see gas vs coal switching depending on which is “less bad” under CO₂.

In [4]:
# What other analysis can you infer from the result?